<a href="https://colab.research.google.com/github/narsym/facial-expression-detection/blob/master/Face_expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing kaggle for download

In [1]:
!pip install kaggle

Upload files to colab

In [ ]:
from google.colab import files
files.upload()

Prepartion to download the dataset

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Download the dataset

In [4]:
!kaggle datasets download -d ashishpatel26/fer2018

 76% 73.0M/96.6M [00:01<00:00, 27.1MB/s]
100% 96.6M/96.6M [00:01<00:00, 57.2MB/s]


Unzip file

In [5]:
!unzip fer2018.zip

Archive:  fer2018.zip
  inflating: README                  
  inflating: fer2013.bib             
  inflating: fer20131.csv            
  inflating: ferSubmission.csv       


Import necessary libraries

In [6]:
import pandas as pd
import tensorflow as tf
import numpy as np

Read the data

In [8]:
data = pd.read_csv('./fer20131.csv')

Check shape

In [9]:
data.shape

(35887, 3)

Check first few rows

In [10]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


Taking training, test and validation data

In [11]:
train_data = data[data.Usage == 'Training']
val_data = data[data.Usage == 'PublicTest']
test_data = data[data.Usage == 'PrivateTest']

Checking the balance of the dataset

In [12]:
train_data.emotion.value_counts()

3    7215
6    4965
4    4830
2    4097
0    3995
5    3171
1     436
Name: emotion, dtype: int64

The records in classes are all not equal, so imbalanced. so use class weights to penalise

In [13]:
zero = train_data.shape[0] // 3995
one = train_data.shape[0] // 436
two = train_data.shape[0] // 4097
three = train_data.shape[0] // 7215
four = train_data.shape[0] // 4830
five = train_data.shape[0] // 3171
six = train_data.shape[0] // 4965

Checking the class weights

In [14]:
zero, one, two, three, four, five, six

(7, 65, 7, 3, 5, 9, 5)

Labels for classes

In [15]:
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


Extracting the data from csv file and converting it into numpy array

In [17]:
def getData(data, ind):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for i in range(data.shape[0]):
            Y.append(int(data.emotion[i + ind]))
            x = [int(p) for p in data.pixels[i + ind].split()]
            x = np.array(x).reshape(48, 48, 1)
            x = tf.image.grayscale_to_rgb(tf.constant(x.tolist())).numpy()
            X.append(x)
    X, Y = np.array(X),  np.array(Y)
    return X, Y

Loading the data

In [18]:
train_x, train_y = getData(train_data, 0)
val_x, val_y = getData(val_data, train_data.shape[0])
test_x, test_y = getData(test_data, train_data.shape[0] + val_data.shape[0])

Checking the shape 

In [21]:
train_x.shape, train_y.shape

((28709, 48, 48, 3), (28709, 7))

Converting the data into dataset object

In [22]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
val_data = tf.data.Dataset.from_tensor_slices((val_x, val_y))
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))

Shuffling the data and making it into batches 

In [23]:
train_data = train_data.batch(64).shuffle(5000)
val_data = val_data.batch(64).shuffle(1000)
test_data = test_data.batch(64).shuffle(1000)

Defining few callbacks

In [42]:
filepath= "./check/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
check = tf.keras.callbacks.ModelCheckpoint(filepath, monitor = 'val_accuracy', verbose = 1, save_best_only = False)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience= 2, verbose=0, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0)
callbacks = [check, reduce_lr]

Class weight dictionary 

In [28]:
class_weight = {0: zero,
                1: one,
                2: two,
                3: three,
                4: four,
                5: five,
                6: six }

Model for the classification

In [37]:
    model = tf.keras.Sequential()
    input_shape = (48,48,3)
    model.add(tf.keras.layers.Conv2D(128, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), activation='relu', padding='same'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(256, (5, 5),activation='relu',padding='same'))
    model.add(tf.keras.layers.Conv2D(256, (5, 5),activation='relu',padding='same'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(512, (3, 3),activation='relu',padding='same'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3),activation='relu',padding='same'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(tf.keras.layers.Conv2D(128, (3, 3),activation='relu',padding='same'))
    model.add(tf.keras.layers.Conv2D(128, (3, 3),activation='relu',padding='same'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(128, activation = 'relu'))
    model.add(tf.keras.layers.Dense(7, activation = 'softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

Training the model

In [ ]:
model.fit(train_data, epochs = 20, callbacks = callbacks, validation_data = val_data)

evaluating on the test set

In [45]:
model.evaluate(test_data)

57/57 [==============================] - 1s 16ms/step - loss: 1.4421 - accuracy: 0.6601


[1.4421006441116333, 0.6600724458694458]

we got 66% on the test set

Saving the model

In [46]:
model.save('./final_model.h5')

Loading the saved model

In [51]:
model2 = tf.keras.models.load_model('./final_model.h5')

Evaluating the loaded model

In [52]:
model2.evaluate(test_data)

57/57 [==============================] - 1s 16ms/step - loss: 1.4421 - accuracy: 0.6601


[1.4421007633209229, 0.6600724458694458]

performance is same.